# Data analysis

Experiment information:
- One million function evaluations
- **sade_remc**: is the best method from HM, but with more evals
- **sade_mc_final**: is sade + MC + ffi9 + rmsd crowding + spicker + hooke jeeves on cluster centroids
- **sade_remc_final**: is the same as above, but REMC instead of MC
- **sade_mc_ffi9_02**: is HM method + forced fragment insertion of size 2 with 0.02 chance of happening per individal per generation
- **sade_remc_ffi9_02**: same as above but with REMC instead of MC

In [1]:
import datetime
import string
import random
import pickle
import time
import sys
import os
import re

import data_utils

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set(style="whitegrid");

In [2]:
root_path = '/home/h3nnn4n/progs/de_supimpa/tools/notebooks/analysis'
base_path = '/home/h3nnn4n/progs/de_supimpa/src'


def reset_path():
    os.chdir(base_path)
    
def reset_to_root_path():
    os.chdir(root_path)

    
reset_to_root_path()
reset_path()

In [3]:
runs = [
    'de_experiment_final',
    'de_sade_remc',
    'de_rosetta',
    'de_ffi'
]

In [4]:
dataset = data_utils.load_all_data(runs)
alldata = data_utils.merge_data(dataset)

INFO: Loaded 4 experiment runs dataset


In [5]:
def pretty_print_experiment_summary(experiment_summary):
    def protein_summary_print(p_data):
        keys = sorted(p_data.keys())

        for key in keys:
            experiment = p_data[key]
            name = key
            data = experiment['data']

            print('%25s    min: %8.2f  mean: %8.2f  std: %8.2f' % (
                name, data['min'], data['mean'], data['std']
            ))
    
    for protein in sorted(experiment_summary.keys()):
        print()
        print(protein.upper())

        protein_summary_print(experiment_summary[protein])

## Experiment Summary with *Best by Energy*

### RMSD data

In [6]:
experiment_summary_rmsd = data_utils.experiment_summary(alldata, mode='best_by_energy', metric='rmsd_after')
pretty_print_experiment_summary(experiment_summary_rmsd)


1AIL
         classic-abinitio    min:     4.75  mean:     7.95  std:     1.56
          sade_mc_ffi9_02    min:     6.37  mean:     8.54  std:     1.15
            sade_mc_final    min:     5.69  mean:    10.15  std:     2.39
                sade_remc    min:     4.26  mean:     8.57  std:     1.42
        sade_remc_ffi9_02    min:     4.46  mean:     8.17  std:     1.36
          sade_remc_final    min:     6.25  mean:    10.55  std:     3.44

1CRN
         classic-abinitio    min:     4.30  mean:     6.57  std:     1.57
          sade_mc_ffi9_02    min:     4.15  mean:     8.60  std:     1.52
            sade_mc_final    min:     5.76  mean:     8.50  std:     1.27
                sade_remc    min:     4.27  mean:     8.74  std:     1.55
        sade_remc_ffi9_02    min:     5.06  mean:     8.50  std:     1.32
          sade_remc_final    min:     4.14  mean:     7.96  std:     1.75

1ENH
         classic-abinitio    min:     2.84  mean:     6.43  std:     1.28
          sade_mc_ff

## Energy data 

In [7]:
experiment_summary_scorefxn = data_utils.experiment_summary(alldata, mode='best_by_energy', metric='scorefxn')
pretty_print_experiment_summary(experiment_summary_scorefxn)


1AIL
         classic-abinitio    min:  -172.76  mean:  -131.39  std:    19.03
          sade_mc_ffi9_02    min:  -153.65  mean:  -121.99  std:    19.02
            sade_mc_final    min:  -160.78  mean:  -138.59  std:    10.18
                sade_remc    min:  -167.59  mean:  -124.49  std:    16.50
        sade_remc_ffi9_02    min:  -160.07  mean:  -131.82  std:    17.00
          sade_remc_final    min:  -166.29  mean:  -134.65  std:    11.21

1CRN
         classic-abinitio    min:   -79.64  mean:   -60.92  std:     8.73
          sade_mc_ffi9_02    min:   -75.67  mean:   -53.17  std:     8.49
            sade_mc_final    min:   -75.86  mean:   -62.55  std:     6.50
                sade_remc    min:   -75.38  mean:   -54.99  std:    12.29
        sade_remc_ffi9_02    min:   -74.92  mean:   -54.47  std:     8.35
          sade_remc_final    min:   -84.79  mean:   -61.62  std:     7.38

1ENH
         classic-abinitio    min:  -138.48  mean:  -104.29  std:    20.83
          sade_mc_ff

## Experiment Summary with *Best by RMSD* 

## RMSD data

In [8]:
experiment_summary_scorefxn = data_utils.experiment_summary(alldata, mode='best_by_rmsd', metric='rmsd_after')
pretty_print_experiment_summary(experiment_summary_scorefxn)


1AIL
         classic-abinitio    min:     4.75  mean:     7.95  std:     1.56
          sade_mc_ffi9_02    min:     6.37  mean:     8.54  std:     1.15
            sade_mc_final    min:     4.92  mean:     7.69  std:     1.22
                sade_remc    min:     4.26  mean:     8.57  std:     1.42
        sade_remc_ffi9_02    min:     4.46  mean:     8.17  std:     1.36
          sade_remc_final    min:     4.79  mean:     7.57  std:     1.14

1CRN
         classic-abinitio    min:     4.30  mean:     6.57  std:     1.57
          sade_mc_ffi9_02    min:     4.15  mean:     8.60  std:     1.52
            sade_mc_final    min:     4.35  mean:     6.82  std:     0.98
                sade_remc    min:     4.27  mean:     8.74  std:     1.55
        sade_remc_ffi9_02    min:     5.06  mean:     8.50  std:     1.32
          sade_remc_final    min:     4.14  mean:     6.44  std:     0.87

1ENH
         classic-abinitio    min:     2.84  mean:     6.43  std:     1.28
          sade_mc_ff

## Energy Data

In [9]:
experiment_summary_scorefxn = data_utils.experiment_summary(alldata, mode='best_by_rmsd', metric='scorefxn')
pretty_print_experiment_summary(experiment_summary_scorefxn)


1AIL
         classic-abinitio    min:  -172.76  mean:  -131.39  std:    19.03
          sade_mc_ffi9_02    min:  -153.65  mean:  -121.99  std:    19.02
            sade_mc_final    min:  -152.81  mean:  -109.20  std:    30.51
                sade_remc    min:  -167.59  mean:  -124.49  std:    16.50
        sade_remc_ffi9_02    min:  -160.07  mean:  -131.82  std:    17.00
          sade_remc_final    min:  -166.29  mean:  -101.15  std:    38.71

1CRN
         classic-abinitio    min:   -79.64  mean:   -60.92  std:     8.73
          sade_mc_ffi9_02    min:   -75.67  mean:   -53.17  std:     8.49
            sade_mc_final    min:   -67.01  mean:   -38.71  std:    24.16
                sade_remc    min:   -75.38  mean:   -54.99  std:    12.29
        sade_remc_ffi9_02    min:   -74.92  mean:   -54.47  std:     8.35
          sade_remc_final    min:   -84.79  mean:   -40.05  std:    30.07

1ENH
         classic-abinitio    min:  -138.48  mean:  -104.29  std:    20.83
          sade_mc_ff